<a href="https://colab.research.google.com/github/vincelemke99/Bachelor-Thesis/blob/main/finales_lead_predication_one_hot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#!pip install pandas category_encoders scikit-learn tensorflow --q
!pip install tensorflow==2.12.0 --q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 585.9/585.9 MB 1.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 75.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.1/17.1 MB 55.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 76.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 440.7/440.7 kB 41.7 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
chex 0.1.86 requires numpy>=1.24.1, but you have numpy 1.23.5 which is incompatible.
pandas-stubs 2.0.3.230814 requires numpy>=1.25.0; python_version >= "3.9", but you have numpy 1.23.5 which is incompatible.
tf-keras 2.15.1 requires tensorflow<2.16,>=2.15, but you have tensorflow 2.12.0 which is incompatible.


In [2]:
# Import necessary libraries
import pandas as pd
import numpy as np
import seaborn as sns
from matplotlib import pyplot as plt
import plotly.express as px
import plotly.graph_objects as go
import plotly.figure_factory as ff

# Import machine learning libraries
import lightgbm as lgb
import xgboost as xgb
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import (
    accuracy_score,
    precision_score,
    recall_score,
    f1_score,
    roc_auc_score,
    roc_curve,
    confusion_matrix
)
from sklearn.model_selection import train_test_split, RandomizedSearchCV, cross_val_score
from sklearn.impute import SimpleImputer
from sklearn.feature_selection import mutual_info_classif

In [3]:
from google.colab import auth
auth.authenticate_user()
print("Authenticated")

MessageError: Error: credential propagation was unsuccessful

In [ ]:
PROJECT_ID = 'prj-snd-ew3-vschmitt'

In [ ]:
from google.cloud import bigquery
client = bigquery.Client(project=PROJECT_ID)

In [ ]:
BQ_PROJECT = PROJECT_ID
BQ_DATASET = 'lead_prediction'
BQ_TABLE = 'trainings_data'

In [ ]:
# SQL query to select data
query = f"""
SELECT *
FROM `{BQ_PROJECT}.{BQ_DATASET}.{BQ_TABLE}`
"""

try:
    # Execute the query and convert the result to a pandas DataFrame
    df = client.query(query=query).to_dataframe()
    print("Query executed successfully and DataFrame is ready.")
except Exception as e:
    print(f"An error occurred: {e}")

In [ ]:
df.shape

In [ ]:
df.head()

In [ ]:
df.drop(['Lead_ID',], axis="columns", inplace=True)

In [ ]:
df.rename(columns={'Lead_Source' : 'Lead Source', 'Lead_Source_Detail' : 'Lead Source Detail',
                   'Name_Case' : 'Name Case', 'has_Phone' : 'has Phone', 'Accept_Data_Policy' : 'Accept Data Policy',
                   'Conversion_Type' : 'Conversion Type', 'Study_Model' : 'Study Model', 'Type_of_Study_Program' : 'Type of Study Program',
                   'Study_Location' : 'Study Location', 'Study_Program' : 'Study Program', 'Sum_of_Study_Interests' : 'Sum of Study Interests' , 'Domain_Type' : 'Domain Type' },inplace=True)


In [ ]:
df.head()

In [ ]:
df.isna().sum().to_frame(name='Missing Values').sort_values('Missing Values', ascending=False)

In [ ]:
# Calculate the percentage of missing values for each column
percentage_missing = df.isnull().sum() * 100 / len(df)
# Create a DataFrame to display the missing values
missing_values_df = percentage_missing.to_frame(name='Missing Values').sort_values('Missing Values', ascending=False)
# Display the DataFrame
missing_values_df

In [ ]:
# Korrelation zwischen den fehlenden Werten
correlation_matrix = df[['Host', 'Conversion Type', 'Gender']].isnull().corr()
sns.heatmap(correlation_matrix, annot=True, cmap='viridis')
plt.title('Korrelation der fehlenden Werte')
plt.show()


In [ ]:
# Histogramme
fig, axes = plt.subplots(1, 3, figsize=(18, 6))

for i, column in enumerate(['Host', 'Conversion Type', 'Gender']):
    sns.histplot(df[column], kde=False, ax=axes[i])
    axes[i].set_title(f'Histogram of {column}')
    axes[i].set_xlabel(column)
    axes[i].set_ylabel('Frequency')

plt.tight_layout()
plt.show()

# Boxplots
fig, axes = plt.subplots(1, 3, figsize=(18, 6))

for i, column in enumerate(['Host', 'Conversion Type', 'Gender']):
    sns.boxplot(y=df[column], ax=axes[i])
    axes[i].set_title(f'Boxplot of {column}')
    axes[i].set_ylabel(column)

plt.tight_layout()
plt.show()

In [ ]:
df['Host'].value_counts()

In [ ]:
unwanted_hosts = ['staging.hs-fresenius.de', 'hsf035.dev.360vier.net', 'b2uxwopd.myraidbox.de', 'digital-health-school.de']

# Filter the DataFrame
df = df[~df['Host'].isin(unwanted_hosts)]

In [ ]:
unwanted_Lead_Source = ['-149307507Publisher', 'Not Found']

# Filter the DataFrame
df = df[~df['Lead Source'].isin(unwanted_Lead_Source )]

In [ ]:
df['Conversion Type'] = df['Conversion Type'].fillna('Infomaterial')

In [ ]:
df['Gender'] = df['Gender'].fillna('weiblich')

In [ ]:
df['Host'] = df['Host'].fillna('www.hs-fresenius.de')

In [ ]:
# Calculate the percentage of missing values for each column
percentage_missing = df.isnull().sum() * 100 / len(df)
# Create a DataFrame to display the missing values
missing_values_df = percentage_missing.to_frame(name='Missing Values').sort_values('Missing Values', ascending=False)
# Display the DataFrame
missing_values_df

In [ ]:
df.describe()

In [ ]:
df.isna().sum().to_frame(name='Missing Values').sort_values('Missing Values', ascending=False)

In [ ]:
plt.figure(figsize=(5,5))
sns.countplot(x='is_converted', data=df)
plt.title('No of Unsuccessful Leads vs Successful Converted Leads', color='blue')
plt.xticks(np.arange(2), ('Unsuccessful', 'Successful'))
plt.xlabel('Conversion Status')
plt.ylabel('Count')
plt.show()

In [ ]:
total_leads = len(df)
successful_leads = df['is_converted'].sum()
conversion_rate = (successful_leads / total_leads) * 100
print(f'Conversion Rate: {conversion_rate:.2f}%')

In [ ]:
# Creating a crosstab DataFrame for demonstration
crosstab_df = pd.crosstab(index=df['Lead Source'], columns='count')

In [ ]:
def plot_frequency_and_percentage(feature, df, category_df, another_row=False, height=8, ylabels=[]):
    if another_row:
        fig, (ax1, ax2) = plt.subplots(nrows=2, ncols=1, figsize=(10, height * 2))
    else:
        fig, (ax1, ax2) = plt.subplots(ncols=2, figsize=(15, 6))
    fig.subplots_adjust(hspace=0.3, wspace=0.3)

    # Get top 5 categories by frequency
    top_categories = category_df[feature].value_counts().nlargest(5).index

    # Frequency plot for top 5 categories
    ax1.set_title(f'Frequency Plot of {feature}', color='blue')
    ax1.set_ylabel(feature)
    ax1.set_xlabel('count')
    sns.countplot(y=feature, data=category_df[category_df[feature].isin(top_categories)],
                  order=top_categories, ax=ax1, color='green')
    if ylabels:
        ax1.set_yticklabels(ylabels)

    # Calculate count of converted leads for top 5 categories
    converted_counts = df[df['is_converted'] == True][feature].value_counts()
    converted_counts = converted_counts[converted_counts.index.isin(top_categories)]

    # Converted leads plot for top 5 categories
    ax2.set_title('Converted Leads Count', color='blue')
    ax2.set_ylabel(feature)
    ax2.set_xlabel('count')
    converted_counts.sort_values().plot(kind='barh', ax=ax2, color='orange')
    if ylabels:
        ax2.set_yticklabels(ylabels)

    # Calculate and display overall conversion rate
    total_leads = len(df)
    successful_leads = df['is_converted'].sum()
    conversion_rate = (successful_leads / total_leads) * 100
    fig.suptitle(f'Overall Conversion Rate: {conversion_rate:.2f}%', fontsize=14, color='red')


plt.show()

In [ ]:

def plot_conversion_rate(df, is_converted_column, feature):
    """
    Plots the conversion rate for a specified feature.

    Parameters:
    df (DataFrame): The dataframe containing the data.
    is_converted_column (str): The name of the column indicating if a conversion occurred.
    feature (str): The feature for which to calculate and plot the conversion rate.
    """
    # Filter data where is_converted is True
    df_converted = df[df[is_converted_column] == True]

    # Calculate the count of conversions for each unique value in the feature
    converted_counts = df_converted[feature].value_counts()

    # Calculate the percentage of each unique value within the converted data
    conversion_rate = (converted_counts / converted_counts.sum()) * 100
    conversion_rate = conversion_rate.sort_values(ascending=False)

    # Plot bar chart
    plt.figure(figsize=(10, 6))
    conversion_rate.plot(kind='bar')
    plt.title(f'Percentage of Converted Users by {feature}')
    plt.xlabel(feature)
    plt.ylabel('Percentage of Converted Users (%)')
    plt.xticks(rotation=45)
    plt.tight_layout()
    plt.show()

In [ ]:
plot_frequency_and_percentage('Lead Source', df, df, True, 6)

In [ ]:
plot_conversion_rate(df, 'is_converted', 'Lead Source')

In [ ]:
#plot_frequency_and_percentage('Lead_Source_Detail', df, df, True, 6)
plot_frequency_and_percentage('Lead Source Detail', df, df, another_row=False, height=8, ylabels=[])

In [ ]:
# Filter for converted leads
converted_leads = df[df['is_converted'] == True]

# Aggregate the data for Lead Source and Lead Source Detail
lead_source_counts = converted_leads['Lead Source'].value_counts()
lead_source_detail_counts = converted_leads['Lead Source Detail'].value_counts()

# Combine the counts into a DataFrame
combined_counts = pd.concat([lead_source_counts, lead_source_detail_counts], axis=1).fillna(0)
combined_counts.columns = ['Lead Source', 'Lead Source Detail']

# Select the top 5 categories overall
top_combined_counts = combined_counts.sum(axis=1).nlargest(5)
top_combined_counts = combined_counts.loc[top_combined_counts.index]

# Create the bar chart
fig, ax = plt.subplots(figsize=(12, 6))

bar_width = 0.35
index = range(len(top_combined_counts))

# Plot Lead Source bars
lead_source_bars = ax.bar(index, top_combined_counts['Lead Source'], bar_width, label='Lead Source', color='skyblue')

# Plot Lead Source Detail bars next to Lead Source bars
lead_source_detail_bars = ax.bar([i + bar_width for i in index], top_combined_counts['Lead Source Detail'], bar_width, label='Lead Source Detail', color='salmon')

# Add labels, title, and legend
ax.set_xlabel('Category')
ax.set_ylabel('Number of Converted Leads')
ax.set_title('Top 5 Lead Sources and Lead Source Details for Converted Leads')
ax.set_xticks([i + bar_width / 2 for i in index])
ax.set_xticklabels(top_combined_counts.index, rotation=45)
ax.legend()

plt.tight_layout()
plt.show()

In [ ]:
plot_frequency_and_percentage('Gender', df, df, True, 6)

In [ ]:
plot_frequency_and_percentage('Study Program', df, df, True, 6)

In [ ]:
plot_frequency_and_percentage('Study Model', df, df, True, 6)

In [ ]:
plot_conversion_rate(df, 'is_converted', 'Study Model')

In [ ]:
plot_frequency_and_percentage('Accept Data Policy', df, df, True, 6)

In [ ]:
plot_conversion_rate(df, 'is_converted', 'Accept Data Policy')

In [ ]:
plot_conversion_rate(df, 'is_converted', 'Name Case')

In [ ]:
plot_frequency_and_percentage('Name Case', df, df, True, 6)

In [ ]:
plot_conversion_rate(df, 'is_converted', 'Domain Type')

In [ ]:
filtered_df = df[df['is_converted'] == True]

# Step 3: Calculate the mean of 's_onTime' for the filtered DataFrame
s_onTime_mean_filtered = filtered_df['Sum of Study Interests'].mean()
print(s_onTime_mean_filtered)

In [ ]:
plot_frequency_and_percentage('has Phone', df, df, True, 6)

In [ ]:
plot_frequency_and_percentage('Name Case', df, df, True, 6)

In [ ]:
plot_frequency_and_percentage('Conversion Type', df, df, True, 6)

In [ ]:
plot_conversion_rate(df, 'is_converted', 'Conversion Type')

In [ ]:
! pip install category_encoders --q

In [ ]:
# Import necessary libraries
import lightgbm as lgb
import xgboost as xgb
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, roc_curve, confusion_matrix
from sklearn.model_selection import train_test_split, RandomizedSearchCV, cross_val_score
from sklearn.preprocessing import LabelEncoder, StandardScaler, OrdinalEncoder
import category_encoders as ce

In [ ]:
df.dtypes

In [ ]:
df.nunique()

In [ ]:

# Step 1: Drop 'is_converted' column from df_encoded and assign to X_
X = df.drop(columns=['is_converted'])

# Step 2: Extract 'is_converted' column from df_encoded and assign to y
y = df['is_converted']

In [ ]:
from sklearn.preprocessing import OneHotEncoder

one_hot_features = ['Host', 'Conversion Type', 'Gender', 'Study Model',  'Faculty', 'Study Location', 'Type of Study Program',
                  'Lead Source', 'Accept Data Policy', 'has Phone', 'Domain Type', 'Name Case', 'Lead Source Detail', 'Study Program', 'Semester']

X = pd.get_dummies(X, columns=one_hot_features)

In [ ]:

# Step 6: Scaling the dataframe
scaler = StandardScaler()
X_scaled = pd.DataFrame(scaler.fit_transform(X), columns=X.columns)

# Step 7: Define the encoded and scaled dataframe as X_prep
X_prep = X_scaled

In [ ]:

lb = LabelEncoder()
lb.fit(y)
y_encoded = lb.transform(y)
print("Encoded labels:",lb.classes_)
y_en = pd.Series(y_encoded)

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_prep, y_en, test_size=0.3, stratify=y, random_state=42)


print("Training Data Shape : ", X_train.shape, y_train.shape)
print("Test Data Shape : ", X_test.shape, y_test.shape)

In [ ]:
from imblearn.over_sampling import SMOTE


# Initialize SMOTENC with Nearest Neighbors Estimator
smote= SMOTE(random_state=42)

# Fit and resample the training data
X_train, y_train = smote.fit_resample(X_train, y_train)


In [ ]:
print("Smote Train Data : ", X_train.shape, y_train.shape)

In [ ]:
# Compute Mutual Information scores
mi_classification = mutual_info_classif(X_train, y_train, discrete_features='auto')

# Create a pandas Series with the MI scores and the feature names
mi_series = pd.Series(mi_classification, index=X_train.columns)

# Exclude the specified column
mi_series = mi_series.drop(labels='Sum of Study Interests')

# Sort the MI scores in descending order and convert to a DataFrame
data_mi = mi_series.sort_values(ascending=False).to_frame(name='MI')

# Display the DataFrame
print(data_mi)


In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# Function to plot the Mutual Information scores
def plot_mi_scores(scores):
    # Select top 10 scores
    top_scores = scores.sort_values(ascending=False).head(30)
    sorted_scores = top_scores.sort_values(ascending=True)
    width = np.arange(len(sorted_scores))
    ticks = sorted_scores.index
    plt.barh(width, sorted_scores)
    plt.yticks(width, ticks)
    plt.title("Top 10 Mutual Information Scores")
    plt.xlabel("Score")
    plt.ylabel("Features")

    # Add text annotations for the bar values
    for index, value in enumerate(sorted_scores):
        plt.text(value, index, f'{value:.2f}', va='center', ha='left')

    plt.show()

# Plot the Mutual Information scores
plt.figure(dpi=100, figsize=(8, 5))
plot_mi_scores(mi_series)



In [ ]:

'''
# Filter data based on Mutual Information scores
filtered_data_mi = data_mi[data_mi['MI'] >= 0.01]

# Ensure indices match with X_train and X_temp columns
common_indices = filtered_data_mi.index.intersection(X_train.columns)
filtered_data_mi = filtered_data_mi.loc[common_indices]

# Apply the filtered indices to X_train and X_temp
X_train = X_train[common_indices]
X_temp = X_temp[common_indices]
'''

In [ ]:
import pandas as pd
import statsmodels.api as sm
from statsmodels.formula.api import ols
from sklearn.feature_selection import f_classif
from scipy import stats

X = df[['Sum of Study Interests']]
y = df['is_converted']

# Apply ANOVA F-test
f_values, p_values = f_classif(X, y)

# Create a DataFrame to display the F-scores and p-values for the specific feature
anova_results = pd.DataFrame({
    'Feature': ['Sum of Study Interests'],
    'F-Score': f_values,
    'p-Value': p_values
})

# Select the feature (in this case, it's already just one)
selected_feature = anova_results.iloc[0]

print("Selected feature based on ANOVA F-test:")
print(selected_feature)

In [ ]:
from collections import Counter
# Verify the distribution
print("Training set distribution:", Counter(y_train))
print("Test set distribution:", Counter(y_test))

In [ ]:
# Function to evaluate models
def evaluate_model(model, X_test, y_test, model_name):
    predictions = model.predict(X_test)
    if hasattr(model, 'predict_proba'):
        proba = model.predict_proba(X_test)[:, 1]
    else:
        proba = model.predict(X_test)

    accuracy = accuracy_score(y_test, predictions)
    precision = precision_score(y_test, predictions)
    recall = recall_score(y_test, predictions)
    f1 = f1_score(y_test, predictions)
    roc_auc = roc_auc_score(y_test, proba)

    cm = confusion_matrix(y_test, predictions)
    tn, fp, fn, tp = cm.ravel()
    specificity = tn / (tn + fp)

    print(f"{model_name} Metrics:")
    print(f"Accuracy: {accuracy}")
    print(f"Precision: {precision}")
    print(f"Recall (Sensitivity): {recall}")
    print(f"F1-Score: {f1}")
    print(f"Specificity: {specificity}")
    print(f"ROC AUC: {roc_auc}")

    fpr, tpr, _ = roc_curve(y_test, proba)
    plt.figure(figsize=(10, 6))
    plt.plot(fpr, tpr, color='blue', label=f"{model_name} (AUC = {roc_auc:.2f})")
    plt.plot([0, 1], [0, 1], color='red', linestyle='--')
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title(f'{model_name} ROC Curve')
    plt.legend()
    plt.show()

    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=['Not Converted', 'Converted'], yticklabels=['Not Converted', 'Converted'])
    plt.xlabel('Predicted')
    plt.ylabel('Actual')
    plt.title(f'{model_name} Confusion Matrix')
    plt.show()

    return {
        'accuracy': accuracy,
        'precision': precision,
        'recall': recall,
        'f1': f1,
        'roc_auc': roc_auc,
        'specificity': specificity
    }

In [ ]:
# Define the DNN model
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.wrappers.scikit_learn import KerasClassifier
# Define the DNN model
def create_dnn_model(input_dim):
    model = Sequential()
    model.add(Dense(32, input_dim=input_dim, activation='relu'))
    model.add(Dense(16, activation='relu'))
    model.add(Dense(1, activation='sigmoid'))
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    return model

In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
import xgboost as xgb
# Pipelines for each model
pipeline_xgb = Pipeline([
    ('clf', xgb.XGBClassifier())
])

pipeline_rf = Pipeline([
    ('clf', RandomForestClassifier())
])

pipeline_dt = Pipeline([
    ('clf', DecisionTreeClassifier())
])

input_dim = X_train.shape[1]
pipeline_dnn = Pipeline([
    ('clf', KerasClassifier(build_fn=lambda: create_dnn_model(input_dim), epochs=50, batch_size=32, verbose=0))
])

In [ ]:
# Train and evaluate models without cross-validation
model_performance = {}

In [ ]:

# Train and evaluate XGBoost
model_performance['XGBoost'] = evaluate_model(pipeline_xgb.fit(X_train, y_train),X_test, y_test, "XGBoost")

In [ ]:

# Train and evaluate RandomForest
model_performance['RandomForest'] = evaluate_model(pipeline_rf.fit(X_train, y_train), X_test, y_test, "RandomForest")

In [ ]:


# Train and evaluate DecisionTree
model_performance['DecisionTree'] = evaluate_model(pipeline_dt.fit(X_train, y_train), X_test, y_test, "DecisionTree")

In [ ]:
model_performance['DNN'] = evaluate_model(pipeline_dnn.fit(X_train, y_train), X_test, y_test, "Deep Neural Network")

In [ ]:
# Print the performance metrics
for model_name, metrics in model_performance.items():
    print(f"\n{model_name} Performance:")
    for metric, value in metrics.items():
        print(f"{metric}: {value:.4f}")

In [ ]:
# Convert the model performance dictionary to a DataFrame
results_df = pd.DataFrame(model_performance).T

# Save results to an Excel file
results_df.to_excel('model_performance_metrics_without_tuning_final.xlsx', index=True)



In [ ]:
from scipy.stats import uniform
param_rand_xgb = {
    'clf__max_depth': [3, 5, 7, 9],
    'clf__learning_rate': uniform(0.001, 0.1),
    'clf__n_estimators': [100, 200, 300],
    'clf__subsample': uniform(0.6, 0.4),
    'clf__colsample_bytree': uniform(0.6, 0.4),
    'clf__gamma': uniform(0, 0.2),
    'clf__min_child_weight': [1, 3, 5],
    'clf__reg_alpha': uniform(0, 0.5),
    'clf__reg_lambda': uniform(1, 1)
}

param_rand_rf = {
    'clf__n_estimators': [100, 200, 300, 500],
    'clf__max_depth': [None, 10, 20, 30],
    'clf__min_samples_split': [2, 5, 10],
    'clf__min_samples_leaf': [1, 2, 4],
    'clf__max_features': ['auto', 'sqrt', 'log2'],
    'clf__bootstrap': [True, False]
}

param_rand_dt = {
    'clf__max_depth': [None, 10, 20, 30],
    'clf__min_samples_split': [2, 5, 10],
    'clf__min_samples_leaf': [1, 2, 4],
    'clf__max_features': ['auto', 'sqrt', 'log2'],
    'clf__bootstrap': [True, False]
}

param_rand_dnn = {
    'clf__num_layers': [1, 2, 3],
    'clf__num_neurons': [16, 32, 64],
    'clf__activation': ['relu', 'tanh'],
    'clf__dropout_rate': uniform(0.1, 0.5),
    'clf__optimizer': ['adam', 'rmsprop'],
    'clf__epochs': [50, 100],
    'clf__batch_size': [32, 64]
}


In [ ]:
from sklearn.model_selection import StratifiedKFold
def perform_randomized_search(pipeline, param_rand, X_train, y_train, n_iter=100):
    cv = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)
    random_search = RandomizedSearchCV(estimator=pipeline, param_distributions=param_rand,
                                       n_iter=n_iter, cv=cv, scoring='roc_auc', n_jobs=-1, verbose=2, random_state=42)
    random_search.fit(X_train, y_train)
    return random_search

In [ ]:
random_search_xgb = perform_randomized_search(pipeline_xgb, param_rand_xgb, X_train, y_train)

In [ ]:
model_performance = {}

In [ ]:
model_performance['XGBoost'] = evaluate_model(random_search_xgb, X_test, y_test, 'XGBoost')

In [ ]:
random_search_rf = perform_randomized_search(pipeline_rf, param_rand_rf, X_train, y_train)

In [ ]:
model_performance['RandomForest'] = evaluate_model(random_search_rf, X_test, y_test, 'RandomForest', X_train, y_train)

In [ ]:
random_search_dt = perform_randomized_search(pipeline_dt, param_rand_dt, X_train, y_train)

In [ ]:
model_performance['DecisionTree'] = evaluate_model(random_search_dt, X_test, y_test, 'DecisionTree', X_train, y_train)

In [ ]:
random_search_dnn = perform_randomized_search(pipeline_dnn, param_rand_dnn, X_train, y_train)

In [ ]:
model_performance['DNN'] = evaluate_model(random_search_dnn, X_test, y_test, "Deep Neural Network")

In [ ]:
# Print the performance metrics
for model_name, metrics in model_performance.items():
    print(f"\n{model_name} Performance:")
    for metric, value in metrics.items():
        print(f"{metric}: {value:.4f}")

In [ ]:
# Print the best parameters
print("\nBest Parameters:")
print(perform_randomized_search.best_params_)

In [ ]:
# Convert the model performance dictionary to a DataFrame
results_df = pd.DataFrame(model_performance).T

# Save results to an Excel file
results_df.to_excel('model_performance_metrics_final.xlsx', index=True)

# Print results for each model
print("Model Performance Metrics:")
print(results_df)
